In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.linalg import lstsq
import scipy.linalg as linalg 
from scipy.linalg import solve
from scipy.stats import norm
import math

# EFD Call,Put and Eurpean, American

In [33]:
def Explicit_X_Put(dt,T,sigma,r,S0,k,K,dx_n):
    X0 = np.log(S0)
    dx = sigma * np.sqrt(dx_n * dt)
    X_max = X0 + k * dx
    X_min = X0 - k * dx
    
    M = np.arange(T / dt).astype(int)
    N = ((X_max - X_min) / dx).astype(int)
    if N % 2 == 0:
        N = np.arange(N+1)
    else:
        N = np.arange(N)
        
    S = np.exp(np.linspace(X_max,X_min,len(N)))

    p_u = dt * ((sigma ** 2) / (2 * dx**2) + (r-sigma**2/2)/(2 * dx))
    p_m = 1 - dt * sigma ** 2 / dx ** 2 - r * dt
    p_d = dt * ((sigma ** 2) / (2 * dx**2) - (r-sigma**2/2)/(2 * dx))
    p = [p_u,p_m,p_d]

    A = pd.DataFrame(np.NaN,index = N, columns = N)
    A.iloc[0,0:3] = p
    A.iloc[-1,len(N)-3:] = p
    A.iloc[1:-1,0:3] = p

    for i in range(2,len(N)-1):
        A.iloc[i] = A.iloc[i].shift(i-1)
    A = np.array(A.fillna(0))

    F_1 = np.maximum(K-S,0)
    payoff = np.maximum(K-S,0)
    # note P6 L6
    F_1[0] = F_1[1]
    F_1[-1] = F_1[-2] + S[-2] - S[-1]
    
    for t in M:
        F = A @ F_1
        F_1[0] = F_1[1]
        # for B
        F[-1] = F[-2] + S[-2] - S[-1]
        # American option; comment out if is European
        F_1 = np.maximum(F,payoff)   
    return F_1[int(len(N)/2)]
Explicit_X_Put(dt= 0.002,T=0.5,sigma=0.25,r=0.055,S0=180,k=50,K=180,dx_n=1)

10.675727249004005

In [52]:
def Explicit_X_Call(dt,T,sigma,r,S0,k,K,dx_n):
    X0 = np.log(S0)
    dx = sigma * np.sqrt(dx_n * dt)
    X_max = X0 + k * dx
    X_min = X0 - k * dx
    
    M = np.arange(T / dt).astype(int)
    N = ((X_max - X_min) / dx).astype(int)
    if N % 2 == 0:
        N = np.arange(N+1)
    else:
        N = np.arange(N)
        
    S = np.exp(np.linspace(X_max,X_min,len(N)))

    p_u = dt * ((sigma ** 2) / (2 * dx**2) + (r-sigma**2/2)/(2 * dx))
    p_m = 1 - dt * sigma ** 2 / dx ** 2 - r * dt
    p_d = dt * ((sigma ** 2) / (2 * dx**2) - (r-sigma**2/2)/(2 * dx))
    p = [p_u,p_m,p_d]

    A = pd.DataFrame(np.NaN,index = N, columns = N)
    A.iloc[0,0:3] = p
    A.iloc[-1,len(N)-3:] = p
    A.iloc[1:-1,0:3] = p

    for i in range(2,len(N)-1):
        A.iloc[i] = A.iloc[i].shift(i-1)
    A = np.array(A.fillna(0))

    F_1 = np.maximum(S-K,0)
    payoff = np.maximum(S-K,0)

    F_1[-1] = F_1[-2]
    F_1[0] = F_1[1] + S[0] - S[1]
    
    for t in M:
        F = A @ F_1
        F[-1] = F[-2]
        F[0] = F[1] + S[0] - S[1]
        # American option; comment out if is European
        F_1 = np.maximum(F,payoff)   
    return F_1[int(len(N)/2)]
Explicit_X_Call(dt= 0.002,T=0.5,sigma=0.25,r=0.055,S0=180,k=200,K=180,dx_n=1)

15.085220219373193

# IFD Call,Put and Eurpean, American

In [46]:
def Implicit_X_Put(dt,T,sigma,r,S0,k,K,dx_n):
    X0 = np.log(S0)
    dx = sigma * np.sqrt(dx_n * dt)
    X_max = X0 + k * dx
    X_min = X0 - k * dx
    
    M = np.arange(T / dt).astype(int)
    N = ((X_max - X_min) / dx).astype(int)
    
    if N % 2 == 0:
        N = np.arange(N+1)
    else:
        N = np.arange(N)
        
    S = np.exp(np.linspace(X_max,X_min,len(N)))
    
    p_u = - dt/2*(sigma ** 2/dx**2 + (r-sigma**2/2)/dx)
    p_m = 1 + dt * sigma ** 2 / dx**2 + r * dt
    p_d = - dt/2*(sigma ** 2/dx**2 - (r-sigma**2/2)/dx)
    p = [p_u,p_m,p_d]
    A = pd.DataFrame(np.NaN,index = N, columns = N)
    A.iloc[0,0:2] = [1,-1]
    A.iloc[-1,len(N)-2:] = [1,-1]
    A.iloc[1:-1,0:3] = p
    for i in range(2,len(N)-1):
        A.iloc[i] = A.iloc[i].shift(i-1)
    A = np.array(A.fillna(0))
    
    Z_1 = np.maximum(K - S,0)
    payoff = np.maximum(K - S,0)
    Z_1[0] = 0
    Z_1[-1] = Z_1[-2] - S[-1] + S[-2]
    for t in M:
        Z_1[0] = 0
        Z_1[-1] = Z_1[-2] - S[-1] + S[-2]
        F = np.linalg.inv(A) @ Z_1
        Z_1 = np.maximum(F,payoff)
    return Z_1[int(len(N)/2)]
Implicit_X_Put(dt= 0.002,T=0.5,sigma=0.25,r=0.055,S0=180,k=100,K=180,dx_n=1)

10.661461966881793

In [53]:
def Implicit_X_Call(dt,T,sigma,r,S0,k,K,dx_n):
    X0 = np.log(S0)
    dx = sigma * np.sqrt(dx_n * dt)
    X_max = X0 + k * dx
    X_min = X0 - k * dx
    
    M = np.arange(T / dt).astype(int)
    N = ((X_max - X_min) / dx).astype(int)
    
    if N % 2 == 0:
        N = np.arange(N+1)
    else:
        N = np.arange(N)
        
    S = np.exp(np.linspace(X_max,X_min,len(N)))
    
    p_u = - dt/2*(sigma ** 2/dx**2 + (r-sigma**2/2)/dx)
    p_m = 1 + dt * sigma ** 2 / dx**2 + r * dt
    p_d = - dt/2*(sigma ** 2/dx**2 - (r-sigma**2/2)/dx)
    p = [p_u,p_m,p_d]
    A = pd.DataFrame(np.NaN,index = N, columns = N)
    A.iloc[0,0:2] = [1,-1]
    A.iloc[-1,len(N)-2:] = [1,-1]
    A.iloc[1:-1,0:3] = p
    for i in range(2,len(N)-1):
        A.iloc[i] = A.iloc[i].shift(i-1)
    A = np.array(A.fillna(0))
    
    B = np.maximum(S - K,0)
    payoff = np.maximum(S - K,0)
    B[-1] = 0
    B[0] = S[0] - S[1]
    
    for t in M:
        F = np.linalg.inv(A) @ B
        F[-1] = 0
        F[0] = S[0] - S[1]
        B = np.maximum(F,payoff)
    return B[int(len(N)/2)]

Implicit_X_Call(dt=0.002,T=0.5,sigma=0.25,r=0.055,S0=180,k=100,K=180,dx_n=1)

15.085143990845683

# Crank Call,Put and Eurpean, American

In [57]:
def Crank_X_Put(dt,T,sigma,r,S0,k,K,dx_n):
    X0 = np.log(S0)
    dx = sigma * np.sqrt(dx_n * dt)
    X_max = X0 + k * dx
    X_min = X0 - k * dx
    
    M = np.arange(T / dt).astype(int)
    N = ((X_max - X_min) / dx).astype(int)
    
    if N % 2 == 0:
        N = np.arange(N+1)
    else:
        N = np.arange(N)
        
    S = np.exp(np.linspace(X_max,X_min,len(N)))

    p_u = - dt/4*(sigma ** 2/dx**2 + (r-sigma**2/2)/dx)
    p_m = 1 + dt * sigma ** 2 / (2*dx**2) + r * dt/2
    p_d = - dt/4*(sigma ** 2/dx**2 - (r-sigma**2/2)/dx)
    p = [p_u,p_m,p_d]

    A = pd.DataFrame(np.NaN,index = N, columns = N)
    A.iloc[0,0:2] = [1,-1]
    A.iloc[-1,len(N)-2:] = [1,-1]
    A.iloc[1:-1,0:3] = [p_u,p_m,p_d]
    
    for i in range(2,len(N)-1):
        A.iloc[i] = A.iloc[i].shift(i-1)
    A = np.array(A.fillna(0))

    p1 = [-p_u,-(p_m-2),-p_d]
    coeff = pd.DataFrame(np.NaN,index = N, columns = N)
    coeff.iloc[0,0:3] = p1
    coeff.iloc[-1,len(N)-3:] = p1
    coeff.iloc[1:-1,0:3] = p1
    
    for i in range(2,len(N)-1):
        coeff.iloc[i] = coeff.iloc[i].shift(i-1)
    coeff = np.array(coeff.fillna(0))
    
    payoff = np.maximum(K - S,0)
    P_1 = np.maximum(K - S,0)
    
    for t in M:
        Z_1 = coeff @ P_1
        Z_1[0] = S[-1] - S[-2]
        Z_1[-1] = 0
        F = np.linalg.inv(A) @ Z_1
        P_1 = np.maximum(F,payoff)
        
    return P_1[int(len(N)/2)]
Crank_X_Put(dt= 0.002,T=0.5,sigma= 0.25,r=0.055,S0=180,k=100,K=180,dx_n=1)

10.672493499667617

In [58]:
def Crank_X_Call(dt,T,sigma,r,S0,k,K,dx_n):
    X0 = np.log(S0)
    dx = sigma * np.sqrt(dx_n * dt)
    X_max = X0 + k * dx
    X_min = X0 - k * dx
    
    M = np.arange(T / dt).astype(int)
    N = ((X_max - X_min) / dx).astype(int)
    
    if N % 2 == 0:
        N = np.arange(N+1)
    else:
        N = np.arange(N)
        
    S = np.exp(np.linspace(X_max,X_min,len(N)))

    p_u = - dt/4*(sigma ** 2/dx**2 + (r-sigma**2/2)/dx)
    p_m = 1 + dt * sigma ** 2 / (2*dx**2) + r * dt/2
    p_d = - dt/4*(sigma ** 2/dx**2 - (r-sigma**2/2)/dx)
    p = [p_u,p_m,p_d]
    A = pd.DataFrame(np.NaN,index = N, columns = N)
    A.iloc[0,0:2] = [1,-1]
    A.iloc[-1,len(N)-2:] = [1,-1]
    A.iloc[1:-1,0:3] = [p_u,p_m,p_d]
    
    for i in range(2,len(N)-1):
        A.iloc[i] = A.iloc[i].shift(i-1)
    A = np.array(A.fillna(0))
    p1 = [-p_u,-(p_m-2),-p_d]
    coeff = pd.DataFrame(np.NaN,index = N, columns = N)
    coeff.iloc[0,0:3] = p1
    coeff.iloc[-1,len(N)-3:] = p1
    coeff.iloc[1:-1,0:3] = p1
    
    for i in range(2,len(N)-1):
        coeff.iloc[i] = coeff.iloc[i].shift(i-1)
    coeff = np.array(coeff.fillna(0))
    
    payoff = np.maximum(S-K,0)
    P_1 = np.maximum(S-K,0)
    
    for t in M:
        Z_1 = coeff @ P_1
        Z_1[0] = 0
        Z_1[-1] = S[-1] - S[-2]
        F = np.linalg.inv(A) @ Z_1
        P_1 = np.maximum(F,payoff)
        
    return P_1[int(len(N)/2)]
Crank_X_Call(dt= 0.002,T=0.5,sigma= 0.25,r=0.055,S0=180,k=100,K=180,dx_n=1)

15.091695941761206